12/11

FAISS, ChromaDB, PGVector에 1000차원 float32 벡터 1000개를 입력한 뒤 쿼리한 결과입니다.
원격으로 연결이 이루어져서 그런지 입력에 시간이 오래걸렸습니다.
별도의 인덱스를 생성하지 않았을 경우, 쿼리시간은 FAISS 2.76e-04, ChromaDB 3.26e-03, PGVector 3.34e-02 순으로 나타났습니다.

1) 입력시간을 줄일 수 있는 방법은 없는지
2) 조회된 값 자체도 차이가 많이 나고 있는데 이유가 무엇인지
2) HNSW 인덱싱을 시도하고, 다양한 차원, 크기에 대해 실험해보았을 때 쿼리시간이 어떻게 나타나는지 조사해보겠습니다.


12/13 업데이트

대량의 데이터 입력시 걸리는 입력시간의 문제는 INSERT가 아닌 COPY 방식으로 해결할 수 있었습니다.
=> 04_copy_loading.ipynb

# 벡터 생성

In [1]:
import psycopg2
import numpy as np
from time import time
from tqdm import tqdm

In [2]:
# 벡터 생성
example_vectors = np.random.randn(1000,1000).astype('float32')
dim_vectors = example_vectors.shape[1]

# 벡터 검색
test = np.random.randn(1, dim_vectors)
test_vec = np.random.randn(1, dim_vectors).astype('float32')  # 검색할 벡터
k = 5  # 상위 5개 유사 벡터 검색

# 연결 및 입력

## FAISS

In [40]:
import os
import numpy as np
import faiss

# FAISS 인덱스 초기화
# L2 거리 (유클리디안 거리) 기준 인덱스 생성
index = faiss.IndexFlatL2(dim_vectors)  # 벡터 차원

# index 생성
example_ids = [f"vec_{i}" for i in range(len(example_vectors))]

# 벡터 삽입
index.add(example_vectors)  # 벡터 추가
print(f"FAISS 인덱스에 {index.ntotal}개의 벡터가 추가되었습니다.")

FAISS 인덱스에 1000개의 벡터가 추가되었습니다.


In [41]:
# ID와 벡터 인덱스 매핑
id_to_index = {i: example_ids[i] for i in range(len(example_vectors))}

# 인덱스 저장
faiss.write_index(index, "faiss_index.bin")

## Chroma

In [42]:
import os
import chromadb
from chromadb.config import Settings
import numpy as np

# 디렉토리 경로
directory = "./chromadb_store"

# 디렉토리 생성
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} 디렉토리가 생성되었습니다.")

# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path=directory)

# 컬렉션 생성 또는 불러오기
collection = client.get_or_create_collection(name="vectors")

In [43]:
# 벡터 삽입
example_ids = [f"vec_{i}" for i in range(len(example_vectors))]

for vec_id, vec in zip(example_ids, example_vectors):
    collection.add(documents=[""], embeddings=[vec.tolist()], ids=[vec_id])

Add of existing embedding ID: vec_0
Insert of existing embedding ID: vec_0
Add of existing embedding ID: vec_1
Insert of existing embedding ID: vec_1
Add of existing embedding ID: vec_2
Insert of existing embedding ID: vec_2
Add of existing embedding ID: vec_3
Insert of existing embedding ID: vec_3
Add of existing embedding ID: vec_4
Insert of existing embedding ID: vec_4
Add of existing embedding ID: vec_5
Insert of existing embedding ID: vec_5
Add of existing embedding ID: vec_6
Insert of existing embedding ID: vec_6
Add of existing embedding ID: vec_7
Insert of existing embedding ID: vec_7
Add of existing embedding ID: vec_8
Insert of existing embedding ID: vec_8
Add of existing embedding ID: vec_9
Insert of existing embedding ID: vec_9
Add of existing embedding ID: vec_10
Insert of existing embedding ID: vec_10
Add of existing embedding ID: vec_11
Insert of existing embedding ID: vec_11
Add of existing embedding ID: vec_12
Insert of existing embedding ID: vec_12
Add of existing emb

## postgres

In [4]:
# 연결
#conn = psycopg2.connect(host='localhost', dbname='postgres', user='admin', password='1234', port=5432)
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# pgvector 확장
query = """
        CREATE EXTENSION IF NOT EXISTS vectors;
        """

cursor.execute(query)

# vector 테이블 생성
cursor.execute("DROP TABLE IF EXISTS vectors;")
cursor.execute(f"CREATE TABLE IF NOT EXISTS vectors (id bigserial PRIMARY KEY, embedding vector({dim_vectors}));")

In [ ]:
cursor.execute("SELECT COUNT(*) from vectors;")
cursor.fetchall()

In [10]:
# 연결
#conn = psycopg2.connect(host='localhost', dbname='postgres', user='admin', password='1234', port=5432)
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

cursor.execute(f"CREATE TABLE IF NOT EXISTS items (id bigserial PRIMARY KEY, embedding vector({dim_vectors}));")


In [12]:
# 벡터 삽입
for vec in tqdm(example_vectors):
    # 벡터 데이터를 삽입 (PostgreSQL의 vector 타입은 문자열 형태로 저장)
    cursor.execute(
        "INSERT INTO items (embedding) VALUES (%s);", 
        (vec.tolist(),)  # numpy 배열을 Python 리스트로 변환
    )

# # 대량 삽입 - 테스트 결과 실행시간은 차이 없음
# vector_data = [(vec.tolist(),) for vec in example_vectors]

# cursor.executemany(
#     "INSERT INTO vectors (embedding) VALUES (%s);",
#     vector_data
# )

# 커밋
conn.commit()
conn.close()

100%|██████████| 1000/1000 [00:25<00:00, 39.82it/s]


# 검색속도비교

In [46]:
# 저장된 인덱스 불러오기
index = faiss.read_index("faiss_index.bin")

# 검색 수행
s = time()
distances, indices = index.search(test_vec, k)
time_spent = time() - s
print(time_spent)

# 결과 출력
def show_results(test_vec, distances, indices):
    for rank, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        vector_id = id_to_index[idx]
        print(f"Rank {rank + 1}: ID = {vector_id}, Distance = {dist}")

print("\nTop 5 Nearest Vectors:")
show_results(test_vec, distances, indices)

0.0002760887145996094

Top 5 Nearest Vectors:
Rank 1: ID = vec_367, Distance = 1677.269775390625
Rank 2: ID = vec_901, Distance = 1705.8570556640625
Rank 3: ID = vec_61, Distance = 1710.5772705078125
Rank 4: ID = vec_558, Distance = 1715.16650390625
Rank 5: ID = vec_443, Distance = 1721.9249267578125


In [47]:

# 컬렉션 로드
# 디렉토리 경로
directory = "./chromadb_store"

# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path=directory)
collection = client.get_or_create_collection(name="vectors")


# 벡터 검색
test_embedding = test_vec.squeeze().tolist()

s = time()
retrieved = collection.query(
    query_embeddings=[test_embedding],
    n_results=5
)
time_spent = time() - s
print(time_spent)

# 결과 출력
def show_results(results):
    for idx, doc_id in enumerate(results["ids"][0]):
        print(f"Rank {idx + 1}: ID = {doc_id}, Similarity = {results['distances'][0][idx]}")

print("\nTop 5 Nearest Vectors:")
show_results(retrieved)

0.0032579898834228516

Top 5 Nearest Vectors:
Rank 1: ID = vec_460, Similarity = 1741.4742431640625
Rank 2: ID = vec_194, Similarity = 1751.2255859375
Rank 3: ID = vec_596, Similarity = 1773.5360107421875
Rank 4: ID = vec_709, Similarity = 1773.861328125
Rank 5: ID = vec_731, Similarity = 1774.7578125


In [5]:
# 연결
#conn = psycopg2.connect(host='localhost', dbname='postgres', user='admin', password='1234', port=5432)
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

In [6]:
# 결과 출력 함수
def show(rows):
    for row in rows:
        print(f"ID: {row[0]}, Similarity: {row[1]}")

# 검색할 벡터 변환
test_vec = test.tolist()[0]

# 유사도를 포함한 쿼리 실행
s = time()
cursor.execute("""
    SELECT id, 
        embedding <-> '{}' AS similarity
    FROM vectors
    ORDER BY similarity
    LIMIT 5
""".format(test_vec))
time_spent = time() - s
print(time_spent)

# 결과 가져오기
results = cursor.fetchall()

show(results)

# 연결 종료
cursor.close()
conn.close()